In [1]:
import sys
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pprint import pprint

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from src.utils.data_utils import drop_id
from src.utils.training_utils import (
    prepare_data,
    train_model,
    hyperparameter_tuning_grid,
    hyperparameter_tuning_random,
    save_model,
    remove_old_models,
    get_models,
)
from src.utils.notebook_setup import setup_notebook_environment

# Quick setup
dbs, logger = await setup_notebook_environment()

# Now ready to work
logger.info("=== STARTING MODEL TRAINING ===")

# Checking Gold Layer
gold_data_from_db = await dbs.get_gold_data()
gold_data_df = pd.DataFrame(gold_data_from_db)
gold_data_df = drop_id(gold_data_df)

gold_data_df.head(10)

2025-07-25 22:20:51,952 - api.services.database_service - INFO - Connected to MongoDB database: healthcare
2025-07-25 22:20:51,957 - src.utils.notebook_setup - INFO - Database connected: True
2025-07-25 22:20:51,958 - src.utils.notebook_setup - INFO - Database collections: ['heart_disease_silver', 'heart_disease_bronze', 'heart_disease_gold']
2025-07-25 22:20:51,959 - src.utils.notebook_setup - INFO - Database collections count: 3
2025-07-25 22:20:51,961 - src.utils.notebook_setup - INFO - === STARTING MODEL TRAINING ===


,age,sex,trestbps,chol,fbs,thalch,exang,oldpeak,cp_asymptomatic,cp_atypical angina,cp_non-anginal,restecg_normal,restecg_st-t abnormality,slope_flat,slope_not_tested,slope_upsloping,target
0,0.714286,1.0,0.541667,0.285714,1.0,0.633803,0.0,0.556818,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.795918,1.0,0.666667,0.388031,0.0,0.338028,1.0,0.465909,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1
2,0.795918,1.0,0.333333,0.277992,0.0,0.485915,1.0,0.590909,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1
3,0.183673,1.0,0.416667,0.318533,0.0,0.894366,0.0,0.693182,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0
4,0.265306,0.0,0.416667,0.229730,0.0,0.788732,0.0,0.454545,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0
5,0.571429,1.0,0.333333,0.291506,0.0,0.830986,0.0,0.386364,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0
6,0.693878,0.0,0.500000,0.353282,0.0,0.704225,0.0,0.704545,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
7,0.591837,0.0,0.333333,0.519305,0.0,0.725352,1.0,0.363636,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0
8,0.714286,1.0,0.416667,0.326255,0.0,0.612676,0.0,0.454545,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1
9,0.510204,1.0,0.500000,0.227799,1.0,0.669014,1.0,0.647727,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [2]:
# SEPARATE features and target
X = gold_data_df.drop('target', axis=1)  # Features only (18 columns)
y = gold_data_df['target']               # Target only (0s and 1s)

# Then do train/test split
X_train, X_test, y_train, y_test = prepare_data(
    X, y, test_size=0.2, random_state=42
)

# scaler = MinMaxScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

logger.info(f"X_train shape: {X_train.shape}")
logger.info(f"X_test shape: {X_test.shape}")
logger.info(f"y_train shape: {y_train.shape}")
logger.info(f"y_test shape: {y_test.shape}")

2025-07-25 22:20:52,080 - src.utils.training_utils - INFO - Training set: (5152, 16), Test set: (1288, 16)
2025-07-25 22:20:52,081 - src.utils.notebook_setup - INFO - X_train shape: (5152, 16)
2025-07-25 22:20:52,082 - src.utils.notebook_setup - INFO - X_test shape: (1288, 16)
2025-07-25 22:20:52,082 - src.utils.notebook_setup - INFO - y_train shape: (5152,)
2025-07-25 22:20:52,084 - src.utils.notebook_setup - INFO - y_test shape: (1288,)


In [3]:
# Define models
models  = get_models(y_train)

pprint(models, indent=4) # Use 4 spaces for inden
pprint(models, width=40) # Set max line width

logger.info(pprint(models))

# Shows ALL available parameters

for model in models:
    logger.info(pprint(f" {model} Parameters: {models[model].get_params()}") ) 
# logger.info(pprint(f" Random Forest Parameters: {models['RandomForest'].get_params()}"))
# logger.info(pprint(f" XGBoost Parameters: {models['XGBoost'].get_params()}"))
# logger.info(pprint(f" Decision Tree Parameters: {models['DecisionTree'].get_params()}"))

2025-07-25 22:20:52,092 - src.utils.training_utils - INFO - 
Retrieved models: dict_keys(['Logistic Regression', 'Random Forest', 'XGBoost', 'SVM'])
2025-07-25 22:20:52,106 - src.utils.notebook_setup - INFO - None
2025-07-25 22:20:52,108 - src.utils.notebook_setup - INFO - None
2025-07-25 22:20:52,109 - src.utils.notebook_setup - INFO - None
2025-07-25 22:20:52,111 - src.utils.notebook_setup - INFO - None
2025-07-25 22:20:52,112 - src.utils.notebook_setup - INFO - None


{   'Logistic Regression': LogisticRegression(max_iter=1000),
    'Random Forest': RandomForestClassifier(class_weight='balanced', random_state=42),
    'SVM': SVC(class_weight='balanced', random_state=42),
    'XGBoost': XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)}
{'Logistic Regression':

In [4]:
# train the models
# Simpler parameter grids
param_grids = {
    'RandomForest': {
        'n_estimators': [50, 100],
        'max_depth': [5, 10, None],
        'min_samples_split': [2, 5]
    },
    'XGBoost': {
        'n_estimators': [50, 100],
        'max_depth': [3, 6],
        'learning_rate': [0.1, 0.2]
    }
}

remove_old_models()
# Train models with correct parameter grids
for name, model in models.items():
    logger.info(f"Training {name}...")
    
    # Train basic model first
    model = train_model(model, X_train, y_train, name)
    
    # Get the correct parameter grid for this model
    if name in param_grids:
        param_grid = param_grids[name]
        logger.info(f"Tuning hyperparameters for {name}...")
        model = hyperparameter_tuning_grid(model, X_train, y_train, param_grid=param_grid, cv=5, n_jobs=-1)
    else:
        logger.info(f"No hyperparameter tuning for {name}")
    
    # Save the model
    save_model(model, name) # TODO: maybe save the model after evaluation

2025-07-25 22:20:52,126 - src.utils.notebook_setup - INFO - Training Logistic Regression...
2025-07-25 22:20:52,165 - src.utils.training_utils - INFO - Trained Logistic Regression
2025-07-25 22:20:52,166 - src.utils.notebook_setup - INFO - No hyperparameter tuning for Logistic Regression
2025-07-25 22:20:52,171 - src.utils.notebook_setup - INFO - Training Random Forest...
2025-07-25 22:20:52,681 - src.utils.training_utils - INFO - Trained Random Forest
2025-07-25 22:20:52,682 - src.utils.notebook_setup - INFO - No hyperparameter tuning for Random Forest
2025-07-25 22:20:52,738 - src.utils.notebook_setup - INFO - Training XGBoost...
2025-07-25 22:20:52,886 - src.utils.training_utils - INFO - Trained XGBoost
2025-07-25 22:20:52,887 - src.utils.notebook_setup - INFO - Tuning hyperparameters for XGBoost...
2025-07-25 22:20:58,968 - src.utils.training_utils - INFO - Hyperparameter tuning completed for XGBClassifier
2025-07-25 22:20:58,975 - src.utils.notebook_setup - INFO - Training SVM...
